## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-12-20-52-25 +0000,missionlocal,ICE arrests 8 asylum seekers at S.F. immigrati...,https://missionlocal.org/2025/09/ice-arrests-8...
1,2025-09-12-20-50-55 +0000,nyt,Bullet Casing in Charlie Kirk Shooting Quoted ...,https://www.nytimes.com/2025/09/12/us/tyler-ro...
2,2025-09-12-20-50-02 +0000,nyt,Charlie Kirk’s Books Make Best-Seller List Aft...,https://www.nytimes.com/live/2025/09/12/us/cha...
3,2025-09-12-20-48-17 +0000,nyt,"As Trump Vows Vengeance, Utah’s Governor Calls...",https://www.nytimes.com/2025/09/12/us/trump-ty...
4,2025-09-12-20-48-13 +0000,startribune,"Vikings QB J.J. McCarthy welcomes first child,...",https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,kirk,57
12,charlie,50
24,trump,33
14,shooting,19
307,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
222,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,193
310,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,189
107,2025-09-12-16-15-38 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,167
249,2025-09-12-06-29-26 +0000,wapo,Officials release new video of person fleeing ...,https://www.washingtonpost.com/nation/2025/09/...,158
43,2025-09-12-19-22-46 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,157


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
222,193,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
310,82,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
249,51,2025-09-12-06-29-26 +0000,wapo,Officials release new video of person fleeing ...,https://www.washingtonpost.com/nation/2025/09/...
231,50,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
46,48,2025-09-12-19-17-06 +0000,nyt,Bolsonaro was Sentenced to 27 Years for Plotti...,https://www.nytimes.com/2025/09/12/world/ameri...
137,41,2025-09-12-14-46-32 +0000,nypost,Trump predicts ‘my little Communist’ Zohran Ma...,https://nypost.com/2025/09/12/us-news/trump-pr...
305,40,2025-09-11-23-19-55 +0000,nypost,Israel strike on Doha ‘killed any hope’ of hos...,https://nypost.com/2025/09/11/world-news/israe...
140,40,2025-09-12-14-43-17 +0000,nypost,Warner Bros. Discovery CEO David Zaslav wants ...,https://nypost.com/2025/09/12/business/warner-...
56,38,2025-09-12-18-43-00 +0000,wsj,Tyler Robinson’s Descent From Promising Studen...,https://www.wsj.com/us-news/who-tyler-robinson...
300,36,2025-09-11-23-55-50 +0000,nyt,Appeals Court Lets Government Cut Off Medicaid...,https://www.nytimes.com/2025/09/11/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
